In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ano = 2018
path_drive_input = f"/content/drive/MyDrive/DataBase/world_bank/to/despesas/input_despesas_to_{ano}"

# Importação

In [ ]:
from typing import IO
import unicodedata
import re
from urllib3 import PoolManager, disable_warnings, exceptions, Timeout, Retry
import urllib.request
from bs4 import BeautifulSoup
import concurrent.futures
import pandas as pd
from more_itertools import batched
import os
import time
from time import sleep
import shutil
from zipfile import ZipFile
import gc
import glob
import requests


class ReponseManage:

  def __init__(self, pool_max_size: int = 1000) -> None:

      http = self.create_pool_manager(pool_max_size)

  def create_pool_manager(self, pool_max_size: int) -> PoolManager:

      http = PoolManager(maxsize=pool_max_size, cert_reqs='CERT_NONE', block=True)
      disable_warnings(exceptions.InsecureRequestWarning)

      return http

class ColetaEntidade:

    def __init__(self, ano: str, id_municipio: str,
                 id_entidade: str, categoria: str, link: str) -> None:

      self.ano = ano
      self.id_municipio = id_municipio
      self.id_entidade = id_entidade
      self.categoria = categoria
      self.link = link


def slugify(s: str):
        s = s.strip().lower()
        s = unicodedata.normalize("NFD", s)
        s = s.encode("ascii", "ignore")
        s = s.decode("utf-8")
        s = s.lower().strip()
        s = re.sub(r'[^\w\s-]', '', s)
        s = re.sub(r'[\s_-]+', '_', s)
        s = re.sub(r'^-+|-+$', '', s)
        return s



def form_link_entidade(pool_manager: PoolManager, id_municipio: int | str, ano: int | str) -> list:

  base_url = "https://portaldocidadao.tce.to.gov.br/estadomunicipios/getUnidades/"

  fields = {
        "municipio": id_municipio,
        "exercicio": ano,
        "remessa": "8",
        "ajax": "true"
    }

  entidades = http.request('GET', base_url, fields=fields).json()

  for entidade in entidades:
    entidade["id_municipio"] = id_municipio
    entidade["ano"] = ano

  return entidades



def form_link_to_data(row: pd.DataFrame,
                      categoria: str = "empenho") -> ColetaEntidade:
  url = f"https://www.tce.to.gov.br/sicap/contabilx/relatorios/planilhas/"
  parametros = f"{categoria}.php?unidade={row.id}&exercicio={row.ano}&bimestre=6&status=1&conferencia=1"
  link = url + parametros
  slot = ColetaEntidade(row.ano, row.id_municipio, row.id, categoria, link)
  return slot


def extrair_entidade(http: PoolManager, row: ColetaEntidade) -> None:

  table = get_table(http, row.link)
  table_save(row, table)

def table_save(row: ColetaEntidade, html: IO) -> None:

  dado = pd.read_html(html, header=0)
  dado = dado[0]
  dado.columns = dado.columns.str.replace(". ", "_").str.lower()
  dado.columns = [slugify(column) for column in dado.columns]

  file_name = f"{row.categoria}.csv"
  base_path = os.path.join(os.getcwd(), "input", row.ano,
                           row.id_municipio,
                           row.id_entidade)

  path_file = os.path.join(base_path, file_name)

  os.makedirs(base_path, exist_ok=True)

  dado.to_csv(path_file, index=False)

  del dado

def download_dado(row: ColetaEntidade):

  file_name = f"{row.categoria}.xls"
  base_path = os.path.join(os.getcwd(), "input", row.ano,
                          row.id_municipio,
                          row.id_entidade)

  path_file = os.path.join(base_path, file_name)

  os.makedirs(base_path, exist_ok=True)
  chunk_size = 1000000 # Define the chunk size (in bytes)

  # Open the URL
  r = requests.get(row.link, stream=True)

  with open(path_file, 'wb') as fd:
      fd.write(r.content)
      for chunk in r.iter_content(chunk_size=chunk_size):
          fd.write(chunk)

def get_table(http: PoolManager, link: str) -> bytes:

    table = http.request('GET', link).data
    return table


def verify_exists(row: pd.DataFrame) -> bool:

    file_name = f"{row.categoria}.xls"
    base_path = os.path.join(os.getcwd(), "input", row.ano,
                            row.id_municipio,
                            row.id_entidade)

    path_file = os.path.join(base_path, file_name)

    path_csv_antigo = str(path_file).replace(".xls", ".csv")

    if not os.path.exists(path_file) and not os.path.exists(path_csv_antigo):
        return True

    return False



def send_folder_drive(path_drive: str, path_input: str) -> None:

  if not len(os.listdir(path_input)) > 0:
    raise Exception("Essa Pasta vazia")
    return None

  shutil.make_archive(path_drive,
                      'zip',
                      path_input)


def send_solo_drive() -> None:

  drive.mount('/content/drive')

  send_folder_drive(path_drive_input + "_auto",
                    "/content/input/")


def extrair(path, path_output) -> None:
  # loading the temp.zip and creating a zip object
  with ZipFile(path, 'r') as zObject:

      # Extracting all the members of the zip
      # into a specific location.
      zObject.extractall(
          path=path_output)


# Verificar IP

In [ ]:
import requests

my_country = requests.get("https://api.myip.com/")
my_country.json()

{'ip': '34.74.110.102', 'country': 'United States', 'cc': 'US'}

# Extrair

In [ ]:
extrair(path=path_drive_input + '.zip',
        path_output="/content/input")

### Extrair Auto-Save

In [ ]:
extrair(path=path_drive_input + "_auto" + '.zip',
        path_output="/content/input")

In [ ]:
from zipfile import ZipFile



# Registrar ID municipios

In [ ]:

http = PoolManager(maxsize=139, cert_reqs='CERT_NONE', block=True)
disable_warnings(exceptions.InsecureRequestWarning)

In [ ]:
link = "https://portaldocidadao.tce.to.gov.br/estadomunicipios/index"
response = http.request('GET', link)

In [ ]:
txt_html = response.data
soup = BeautifulSoup(txt_html, 'html.parser')

In [ ]:
elementos_municipios = soup.select('select[name="comboMunicipio"] option')[1:]
anos = soup.select('select[name="comboExercicio"] option')[1:]
anos = [ano.get("value") for ano in anos]

In [ ]:
dataframe = {elemento.text: elemento.get("value") for elemento in elementos_municipios}

entidades = []
for ano in anos:
    with concurrent.futures.ThreadPoolExecutor() as executor:
      entidades += [entidades_json
                    for entidades_json in
                    executor.map(
                    lambda id_municipio: form_link_entidade(http, id_municipio, ano),
                    dataframe.values()
                    )]

In [ ]:
entidades_to_df = [entidade for entidades_list in entidades for entidade in entidades_list]
df_entindade = pd.DataFrame(entidades_to_df, dtype=str)
df_entindade.columns = df_entindade.columns.str.lower()
df_entindade.to_csv("entidades_to.csv", index=False)

In [ ]:
entidade_json = form_link_entidade(http, dataframe["Abreulândia"], 2023)

In [ ]:
dataframe = {elemento.text: elemento.get("value") for elemento in elementos_municipios}
dataframe

In [ ]:
entidade_json

In [ ]:
links_entidades = []
"https://portaldocidadao.tce.to.gov.br/estadomunicipios/getUnidades?municipio=1700400&exercicio=2023&remessa=8&ajax=true"

In [ ]:
"https://www.tce.to.gov.br/sicap/contabilx/relatorios/planilhas/empenho.php?unidade=12099581000140&exercicio=2023&bimestre=6&status=1&conferencia=1"
empenho = "https://www.tce.to.gov.br/sicap/contabilx/relatorios/planilhas/empenho.php?unidade=01800242000122&exercicio=2023&bimestre=6&status=1&conferencia=1"
liquidaco = "https://www.tce.to.gov.br/sicap/contabilx/relatorios/planilhas/liquidacao.php?unidade=01800242000122&exercicio=2023&bimestre=6&status=1&conferencia=1"
pagamentos = "https://www.tce.to.gov.br/sicap/contabilx/relatorios/planilhas/pagamento.php?unidade=01800242000122&exercicio=2023&bimestre=6&status=1&conferencia=1"
# response = http.request('GET', link)

In [ ]:
response

Link to pandas

In [ ]:
import pandas as pd

In [ ]:
extract_dados()

In [ ]:
empenho

# Raspagem

In [ ]:
entidades = pd.read_csv("/content/drive/MyDrive/DataBase/world_bank/to/info_base/entidades_to.csv", dtype=str)

In [ ]:
chucksize = 200

# retries = Retry(total=0)
# timeout = Timeout(total=40)
# http = PoolManager(maxsize=chucksize, cert_reqs='CERT_NONE', block=True,
#                    timeout=timeout, retries=retries)

# disable_warnings(exceptions.InsecureRequestWarning)
# disable_warnings()
entidade_ano_coleta = entidades[entidades.ano == str(ano)]
categorias = ["empenho", "liquidacao", "pagamento"]
linha_coleta = [form_link_to_data(entidade, categoria)
                for entidade in entidade_ano_coleta.itertuples()
                for categoria in categorias]

chunks = list(batched(linha_coleta, chucksize))

In [ ]:
for n, chunk in enumerate(chunks):

  try:

    start_time = time.time()
    print(f"Chunk: {n+1}/{len(chunks)} Iniciado\n")

    rows_to_extract = [row for row in chunk if verify_exists(row)]

    if rows_to_extract:

      with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(download_dado, rows_to_extract)

      # if (n + 1) % 10 == 0:
      #   send_solo_drive()

      sleep(1)

    print(f"Duração da execução: {(time.time() - start_time) / 60:.2f}\n{'-' * 30}")


  except Exception as erro:
    print(f"Erro: {erro}\nFalha chunk {n+1}")
    sleep(10)


# send_solo_drive()


# Numero de arquivo

In [ ]:
entidades.ano.value_counts()

ano
2024    963
2023    956
2022    937
2021    915
2020    904
2019    884
2018    814
2017    733
2016    686
2015    685
2014    657
2013    597
Name: count, dtype: int64

In [ ]:
path = f"/content/input/{ano}/**/**/*"
files = glob.glob(path)
len(files)

1971

In [ ]:
len(entidade_ano_coleta.id.unique())

657

In [ ]:
entidade_ano_coleta.shape[0] * 3

1971

# Verificar dados perdidos

In [ ]:
path = f"/content/input/{ano}/**/**/*"
files = glob.glob(path)

ids_entidades_coletados = [path.split("/")[-2] for path in files]
ids_entidades_coletados = list(dict.fromkeys(ids_entidades_coletados))

mask = ~entidade_ano_coleta.id.isin(ids_entidades_coletados)
nao_coletados = entidade_ano_coleta[mask]

categorias = ["empenho", "liquidacao", "pagamento"]
nao_coletados = [form_link_to_data(entidade, categoria)
                for entidade in nao_coletados.itertuples()
                for categoria in categorias]

for row in nao_coletados:
  print(row.link)

# Verificar Lucas

In [ ]:
entidades = pd.read_csv("/content/drive/MyDrive/DataBase/world_bank/to/info_base/entidades_to.csv", dtype=str)

In [ ]:
id_municipio = "1718204"

In [ ]:
for nome, id_ in dataframe.items():
  if id_ == id_municipio:
    print(nome, id_)

Porto Nacional 1718204


In [ ]:
marks = (entidades.id_municipio == id_municipio) & (entidades.ano == str(ano))
entidades[marks]

In [ ]:
def ler_html(path: str) -> pd.DataFrame:
  dado = pd.read_html(path, header=0, thousands='.',decimal=',')
  dado = dado[0]
  dado.columns = dado.columns.str.replace(". ", "_").str.lower()
  dado.columns = [slugify(column) for column in dado.columns]
  return dado

In [ ]:
# df1 = ler_html("/content/input/2017/1718006/11847777000102/empenho.xls")
# df2 = ler_html("/content/input/2017/1718006/11847777000102/liquidacao.xls")

ano = 2018
id_municipio = 1700251
df1 = ler_html(f"/content/input/2018/1700251/11291277000137/empenho.xls")
# df2 = ler_html(f"/content/input/{ano}/{id_municipio}/04244263000105/empenho.xls")

In [ ]:
positivo = df1[df1.sinal == "+"].valor.sum()
negativo = df1[df1.sinal == "-"].valor.sum()

In [ ]:
positivo - negativo

11247619.68

In [ ]:
df2 == df1

In [ ]:
correcao = ler_html("https://www.tce.to.gov.br/sicap/contabilx/relatorios/planilhas/empenho.php?unidade=04244263000105&exercicio=2017&bimestre=6&status=1&conferencia=1")

In [ ]:
correcao

In [ ]:
correcao2 = ler_html("https://www.tce.to.gov.br/sicap/contabilx/relatorios/planilhas/empenho.php?unidade=04244263000105&exercicio=2017&bimestre=6&status=1&conferencia=1")

In [ ]:
correcao2

# Mandar para o drive Manualmente

In [ ]:
drive.mount('/content/drive')


send_folder_drive(path_drive_input,
                f"/content/input/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
